In [148]:
import numpy as np
import math
import Bio.PDB
import numpy
from Bio.PDB.PDBParser import PDBParser
import os
import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm as cm
import random
from utils import *


In [14]:
huckel_flag = 1; 
#tert_frust_mode = "configurational";
tert_frust_mode = "mutational";
tert_frust_cutoff= 9.5
tert_frust_ndecoys = 1000; 


In [4]:
with open('water.coeff', 'r') as f:
    k_water = float(f.readline().strip())
    water_kappa, water_kappa_sigma = map(float, f.readline().strip().split())
    threshold = float(f.readline().strip())
    contact_cutoff = float(f.readline().strip())
    n_wells = int(f.readline().strip())

    well_r_min = [0] * n_wells
    well_r_max = [0] * n_wells
    well_flag = [0] * n_wells
    for j in range(n_wells):
        well_r_min[j], well_r_max[j], well_flag[j] = map(float, f.readline().strip().split())

def compute_water_energy(rij, ires_type, jres_type, rho_i, rho_j):
    water_gamma_0_direct = get_water_gamma(0, ires_type, jres_type, 0)
    water_gamma_1_direct = get_water_gamma(0, ires_type, jres_type, 1)

    water_gamma_prot_mediated = get_water_gamma(1, ires_type, jres_type, 0)
    water_gamma_wat_mediated = get_water_gamma(1, ires_type, jres_type, 1)

    sigma_wat = 0.25 * (1.0 - math.tanh(water_kappa_sigma * (rho_i - threshold))) * \
                (1.0 - math.tanh(water_kappa_sigma * (rho_j - threshold)))
    sigma_prot = 1.0 - sigma_wat

    sigma_gamma_direct = (water_gamma_0_direct + water_gamma_1_direct) / 2
    sigma_gamma_mediated = sigma_prot * water_gamma_prot_mediated + sigma_wat * water_gamma_wat_mediated

    t_min_direct = math.tanh(water_kappa * (rij - well_r_min[0]))
    t_max_direct = math.tanh(water_kappa * (well_r_max[0] - rij))
    theta_direct = 0.25 * (1.0 + t_min_direct) * (1.0 + t_max_direct)

    t_min_mediated = math.tanh(water_kappa * (rij - well_r_min[1]))
    t_max_mediated = math.tanh(water_kappa * (well_r_max[1] - rij))
    theta_mediated = 0.25 * (1.0 + t_min_mediated) * (1.0 + t_max_mediated)

    water_energy = -(sigma_gamma_direct * theta_direct + sigma_gamma_mediated * theta_mediated)
    return water_energy


In [5]:
with open('burial.coeff', 'r') as f:
    k_burial = float(f.readline())
    burial_kappa = float(f.readline())
    burial_ro_min = [0.0, 0.0, 0.0]
    burial_ro_max = [0.0, 0.0, 0.0]
    burial_ro_min[0], burial_ro_max[0] = map(float, f.readline().split())
    burial_ro_min[1], burial_ro_max[1] = map(float, f.readline().split())
    burial_ro_min[2], burial_ro_max[2] = map(float, f.readline().split())

def compute_burial_energy(ires_type, rho_i):
    t = [[0.0, 0.0] for _ in range(3)]
    burial_gamma = [0.0, 0.0, 0.0]
    burial_energy = 0.0
    
    t[0][0] = math.tanh(burial_kappa * (rho_i - burial_ro_min[0]))
    t[0][1] = math.tanh(burial_kappa * (burial_ro_max[0] - rho_i))
    t[1][0] = math.tanh(burial_kappa * (rho_i - burial_ro_min[1]))
    t[1][1] = math.tanh(burial_kappa * (burial_ro_max[1] - rho_i))
    t[2][0] = math.tanh(burial_kappa * (rho_i - burial_ro_min[2]))
    t[2][1] = math.tanh(burial_kappa * (burial_ro_max[2] - rho_i))

    burial_gamma[0] = get_burial_gamma(ires_type, 0)
    burial_gamma[1] = get_burial_gamma(ires_type, 1)
    burial_gamma[2] = get_burial_gamma(ires_type, 2)

    burial_energy += -0.5 * k_burial * burial_gamma[0] * (t[0][0] + t[0][1])
    burial_energy += -0.5 * k_burial * burial_gamma[1] * (t[1][0] + t[1][1])
    burial_energy += -0.5 * k_burial * burial_gamma[2] * (t[2][0] + t[2][1])

    return burial_energy    

In [6]:
with open('huckel.coeff', 'r') as f:
    k_PlusPlus, k_MinusMinus, k_PlusMinus = map(float, f.readline().split())
    k_screening = float(f.readline())
    screening_length = float(f.readline())
    debye_huckel_min_sep = int(f.readline())

def compute_electrostatic_energy(rij, i_resno, j_resno, i_resname, j_resname):
    if abs(i_resno - j_resno) < debye_huckel_min_sep:
        return 0.0
    charge_i = 0.0
    charge_j = 0.0
    term_qq_by_r = 0.0
    # Check if ires_type is D, E, R, or K; if not, skip; if so, assign charge type
    if i_resname == 'R' or i_resname == 'K':
        charge_i = 1.0
    elif i_resname == 'D' or i_resname == 'E':
        charge_i = -1.0
    else:
        return 0.0
    # Check if jres_type is D, E, R, or K; if not, skip; if so, assign charge type
    if j_resname == 'R' or j_resname == 'K':
        charge_j = 1.0
    elif j_resname == 'D' or j_resname == 'E':
        charge_j = -1.0
    else:
        return 0.0
    if charge_i > 0.0 and charge_j > 0.0:
        term_qq_by_r = k_PlusPlus * charge_i * charge_j / rij
    elif charge_i < 0.0 and charge_j < 0.0:
        term_qq_by_r = k_MinusMinus * charge_i * charge_j / rij
    else:
        term_qq_by_r = k_PlusMinus * charge_i * charge_j / rij

    return  term_qq_by_r * math.exp(-k_screening * rij / screening_length)

In [7]:
def compute_native_ixn(rij, i_resno, j_resno, resname_list, rho, dist_matrix):
    i_resname = resname_list[i_resno];
    j_resname = resname_list[j_resno];
    reslen = len(rho);
    rho_i = rho[i_resno]
    rho_j = rho[j_resno]
    ires_type  = get_residue_type(i_resname)
    jres_type  = get_residue_type(j_resname)
    water_energy = compute_water_energy(rij, ires_type, jres_type, rho_i, rho_j)
    burial_energy_i = compute_burial_energy(ires_type, rho_i)
    burial_energy_j = compute_burial_energy(jres_type, rho_j)

    electrostatic_energy = 0
    if huckel_flag:
        electrostatic_energy = compute_electrostatic_energy(rij, i_resno, j_resno, i_resname, j_resname)

    if tert_frust_mode == "configurational":
        return water_energy + burial_energy_i + burial_energy_j + electrostatic_energy
    elif tert_frust_mode == "mutational":
        for k in range(reslen):
            if k == i_resno or k == j_resno:
                continue
            rho_k = rho[k]
            k_resname = resname_list[k]
            kres_type = get_residue_type(k_resname)
            rik = dist_matrix[i_resno, k]
            if rik < tert_frust_cutoff:
                water_energy += compute_water_energy(rik, ires_type, kres_type, rho_i, rho_k)
            if huckel_flag:
                electrostatic_energy += compute_electrostatic_energy(rik, i_resno, k, i_resname, k_resname)
            rjk = dist_matrix[j_resno, k]
            if rjk < tert_frust_cutoff:
                water_energy += compute_water_energy(rjk,  jres_type, kres_type, rho_j, rho_k)
            if huckel_flag:
                electrostatic_energy += compute_electrostatic_energy(rjk, j_resno, k, j_resname, k_resname)
        return water_energy + burial_energy_i + burial_energy_j + electrostatic_energy


In [16]:
def compute_decoy_ixns(i_resno, j_resno, resname_list, rho, dist_matrix):
    reslen = len(rho);
    rij_orig = dist_matrix[i_resno, j_resno];
    rho_i_orig = rho[i_resno];
    rho_j_orig = rho[j_resno];
    tert_frust_decoy_energies = np.zeros(tert_frust_ndecoys);
    for decoy_i in range(tert_frust_ndecoys):
        if tert_frust_mode == "configurational":
            rand_i_resno = random.randint(0, reslen - 1)
            rand_j_resno = random.randint(0, reslen - 1)
            rij = dist_matrix[rand_i_resno, rand_j_resno]
            while rij > tert_frust_cutoff or rand_i_resno == rand_j_resno:
                rand_i_resno = random.randint(0, reslen - 1)
                rand_j_resno = random.randint(0, reslen - 1)
                rij = dist_matrix[rand_i_resno, rand_j_resno]
            rand_i_resno_burial = random.randint(0, reslen - 1)
            rand_j_resno_burial = random.randint(0, reslen - 1)
            rho_i = rho[rand_i_resno_burial]
            rho_j = rho[rand_j_resno_burial]
        else:
            rij = rij_orig
            rho_i = rho_i_orig
            rho_j = rho_j_orig

        rand_i_resno = random.randint(0, reslen - 1)
        rand_j_resno = random.randint(0, reslen - 1)
        i_resname = resname_list[rand_i_resno]; 
        j_resname = resname_list[rand_j_resno]
        ires_type = get_residue_type(i_resname)
        jres_type = get_residue_type(j_resname)

        water_energy = compute_water_energy(rij, ires_type, jres_type, rho_i, rho_j)
        burial_energy_i = compute_burial_energy(ires_type, rho_i)
        burial_energy_j = compute_burial_energy(jres_type, rho_j)
        electrostatic_energy = compute_electrostatic_energy(rij, rand_i_resno, rand_j_resno, i_resname, j_resname) if huckel_flag else 0.0

        if tert_frust_mode == "mutational":
            for k in range(reslen):
                if k != i_resno and k != j_resno:
                    rho_k = rho[k]
                    k_resname = resname_list[k]
                    kres_type = get_residue_type(k_resname)
                    rik = dist_matrix[i_resno, k]
                    if rik < tert_frust_cutoff:
                        water_energy += compute_water_energy(rik, ires_type, kres_type, rho_i, rho_k)
                        electrostatic_energy += compute_electrostatic_energy(rik, rand_i_resno, k, i_resname, k_resname) if huckel_flag else 0.0
                    rjk = dist_matrix[j_resno, k]
                    if rjk < tert_frust_cutoff:
                        water_energy += compute_water_energy(rjk, jres_type, kres_type, rho_j, rho_k)
                        electrostatic_energy += compute_electrostatic_energy(rjk, rand_j_resno, k, j_resname, k_resname) if huckel_flag else 0.0

        tert_frust_decoy_energies[decoy_i] = water_energy + burial_energy_i + burial_energy_j + electrostatic_energy
    return np.mean(tert_frust_decoy_energies), np.std(tert_frust_decoy_energies)
    if tert_frust_mode == "configurational":
        already_computed_configurational_decoys = 1

In [156]:
def get_index_water(pdbcode):
    se_map = ["ALA", "ARG", "ASN", "ASP", "CYS", "GLN", "GLU", "GLY", "HIS", "ILE", "LEU", "LYS", "MET", "PHE", "PRO", "SER", "THR", "TRP", "TYR", "VAL", "MSE"]
    se_map_b = ["A", "R", "N", "D", "C", "Q", "E", "G", "H", "I", "L", "K", "M", "F", "P", "S", "T", "W", "Y", "V", "M"]
    p = PDBParser(PERMISSIVE=1)
    s = p.get_structure(pdbcode, pdbcode+'.pdb')
    #chains = s[0].get_list()
    chains = s[0].get_list()
    ca_atoms = [];
    cb_atoms = [];
    cid_list = [];
    resname_list = [];
    #residue_one[atom_map[se_map.index(residue_one.get_resname())]];
    for chain in chains:
        for res in chain:
            cid_list.append(chain.id+str(res.get_id()[1]));
            #print res.get_id()[1]
            if (res.get_resname() in se_map) and (res.get_resname() == 'GLY' or res.has_id('CB')==0):
                ca_atoms.append(res['CA'].get_coord())
                resname_list.append(se_map_b[se_map.index(res.get_resname())])
            if (res.get_resname() in se_map) and res.has_id('CB'):
                ca_atoms.append(res['CB'].get_coord())
                resname_list.append(se_map_b[se_map.index(res.get_resname())])
            if 'CA' in res:
                cb_atoms.append(res['CA'].get_coord())
    dist_matrix = calc_dist_matrix(ca_atoms);
    return dist_matrix, cid_list, resname_list, cb_atoms

def compute_tert_frust(pdbcode):
    atomselect = 0; 
    already_computed_configurational_decoys = 0;
    dist_matrix, cid_list, resname_list, ca_atoms = get_index_water(pdbcode);
    reslen = len(resname_list);
    rho = calc_rho(reslen, dist_matrix);
    tert_frust_output_file = open('tertiary_frustration.dat', 'w')
    tert_frust_vmd_script = open('tertiary_frustration.tcl', 'w')

    ##pymol script
    fpml = open('tertiary_frustration.pml', 'w')
    fpml.write("hide all\n")
    fpml.write("unset dynamic_measures\n")
    fpml.write("show cartoon, all\n")
    fpml.write("color grey, all\n")
    fpml.write("run draw_links.py\n")
    # Double loop over all residue pairs
    for i in range(reslen):
        # get information about residue i
        i_resno = int(cid_list[i][1:])-1; 
        i_resname = resname_list[i]
        ires_type = get_residue_type(i_resname)
        i_chno = cid_list[i][0]
        for j in range(i + 1, reslen):
            # get information about residue j
            j_resno = int(cid_list[j][1:])-1
            j_resname = resname_list[j]
            jres_type = get_residue_type(j_resname)
            j_chno = cid_list[j][0]
            # get the distance between i and j
            rij = dist_matrix[i_resno, j_resno]
            if (rij < tert_frust_cutoff and abs(i - j) >= 2) or (rij < tert_frust_cutoff and i_chno != j_chno):
                #print(rij, i, j, 'filtered')
                rho_i = rho[i_resno]
                rho_j = rho[j_resno]
                native_energy = compute_native_ixn(rij, i_resno, j_resno, resname_list, rho, dist_matrix)
                if tert_frust_mode != "configurational" or (tert_frust_mode == "configurational" and not already_computed_configurational_decoys):
                    e_mean, e_std = compute_decoy_ixns(i_resno, j_resno, resname_list, rho, dist_matrix);
                frustration_index = (native_energy - e_mean) / e_std; 
                # write information out to output file
                tert_frust_output_file.write(f"{i_resno + 1:>5} {j_resno + 1:>5} {i_chno} {j_chno} {rho_i:>8.3f} {rho_j:>8.3f} {resname_list[i_resno]} {resname_list[j_resno]} {native_energy:>8.3f} {e_mean:>8.3f} {e_std:>8.3f} {frustration_index:>8.3f} {ca_atoms[i][0]:>8.3f} {ca_atoms[i][1]:>8.3f} {ca_atoms[i][2]:>8.3f} {ca_atoms[j][0]:8.3f} {ca_atoms[j][1]:>8.3f} {ca_atoms[j][2]:>8.3f}\n")
                if frustration_index < -0.78 or frustration_index > 1:
                    # write information out to vmd script
                    tert_frust_vmd_script.write(f"set sel{i_resno} [atomselect top \"resid {i_resno + 1} and name CA\"]\n")
                    tert_frust_vmd_script.write(f"set sel{j_resno} [atomselect top \"resid {j_resno + 1} and name CA\"]\n")
                    tert_frust_vmd_script.write(f"lassign [atomselect{atomselect} get {{x y z}}] pos1\n")
                    atomselect += 1
                    tert_frust_vmd_script.write(f"lassign [atomselect{atomselect} get {{x y z}}] pos2\n")
                    atomselect += 1
                    if frustration_index < -0.78:
                        tert_frust_vmd_script.write("draw color green\n")
                        fpml.write("draw_links resi " + str(i_resno+1) + " and name CA" + " and Chain " + i_chno + ", resi " + str(j_resno+1) +  " and name CA" + " and Chain " + j_chno + ", color=green, color2=green, radius=0.05, object_name=" + str(i_resno+1) +":"+ str(j_resno+1) + "_green" + i_chno + j_chno +  "\n" );

                    else:
                        tert_frust_vmd_script.write("draw color red\n");
                        fpml.write("draw_links resi " + str(i_resno+1) + " and name CA" + " and Chain " + i_chno + ", resi " + str(j_resno+1) +  " and name CA" + " and Chain " + j_chno + ", color=red, color2=red, radius=0.05, object_name=" + str(i_resno+1) +":"+ str(j_resno+1) + "red" + i_chno + j_chno +  "\n" );

                    if rij < well_r_max[0]:
                        tert_frust_vmd_script.write("draw line $pos1 $pos2 style solid width 1\n")
                    else:
                        tert_frust_vmd_script.write("draw line $pos1 $pos2 style dashed width 2\n")

    # after looping over all pairs, write out the end of the vmd script
    tert_frust_vmd_script.write("mol modselect 0 top \"all\"\n")
    tert_frust_vmd_script.write("mol modstyle 0 top newcartoon\n")
    tert_frust_vmd_script.write("mol modcolor 0 top colorid 15\n")
    tert_frust_vmd_script.close()
    tert_frust_output_file.close()
    fpml.write('select ligand, organic\n')
    fpml.write('select inorganiclig, inorganic\n')
    fpml.write('show spheres, inorganiclig\n')
    fpml.write('show sticks, ligand\n')
    fpml.write('color yellow, ligand\n')
    fpml.write('color magenta, inorganiclig\n')
    fpml.write("zoom all\n");
    fpml.write("hide labels\n");
    fpml.close()

In [157]:
compute_tert_frust('1R69')

In [158]:
def visualize(pdbcode, datfile):
    view = py3Dmol.view()
    view.addModel(open(pdbcode + '.pdb', 'r').read(),'pdb')
    #view.addSurface(py3Dmol.VDW,{'opacity':0.4,'color':'white'}, chA)
    view.setBackgroundColor('white')
    view.setStyle({'chain':'A'}, {'cartoon': {'color':'white'}})

    df = pd.read_csv('tertiary_frustration.dat', header=None, sep = "\s+")
    count_high = 0; 
    count_low = 0; 
    for i in range(len(df)):
        if df.iloc[i, 11] > 1.0 or df.iloc[i, 11] < -0.78:
            start_coord = [df.iloc[i, 12], df.iloc[i, 13], df.iloc[i, 14]]
            end_coord = [df.iloc[i, 15], df.iloc[i, 16], df.iloc[i, 17]]
            if df.iloc[i, 11] > 1.0:
                count_high += 1; 
                view.addCylinder({
                        'start': {'x': start_coord[0], 'y': start_coord[1], 'z': start_coord[2]},
                        'end': {'x': end_coord[0], 'y': end_coord[1], 'z': end_coord[2]},
                        'radius': 0.08,
                        'fromCap': 1,
                        'toCap': 1,
                        'color': 'red',
                        'dashes': True
                })
            else:
                count_low += 1; 
                view.addCylinder({
                        'start': {'x': start_coord[0], 'y': start_coord[1], 'z': start_coord[2]},
                        'end': {'x': end_coord[0], 'y': end_coord[1], 'z': end_coord[2]},
                        'radius': 0.08,
                        'fromCap': 1,
                        'toCap': 1,
                        'color': 'green',
                        'dashes': True
                })
    view.render()
    view.zoomTo()
    view.show()
    print(count_high, count_low)
visualize('1R69', 'tertiary_frustration.dat')

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

39 142


In [62]:
dict(x=ca_atoms[1][0],y=ca_atoms[1][1],z=ca_atoms[1][2])
dict(x=ca_atoms[5][0] ,y=ca_atoms[5][1],z=ca_atoms[5][2])

{'x': -11.275, 'y': -0.973, 'z': 22.291}